In [ ]:
import requests as req
from bs4 import BeautifulSoup as bs
from datetime import datetime
from datetime import datetime, timedelta
import csv
import os

# Headers untuk menghindari pemblokiran
hades = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

In [ ]:
# Tentukan rentang waktu pencarian
start_date = '2020/01/01'  # Tanggal mulai
end_date = '2025/01/01'    # Tanggal akhir

In [ ]:
def scrape_cnbc(hal):
    a = 1
    # Nama file CSV
    csv_file = 'PT TLKM.csv'

    # Cek apakah file CSV sudah ada
    file_exists = os.path.isfile(csv_file)

    for page in range(1, hal):
        # URL untuk mencari artikel dengan kata kunci tertentu dan rentang tanggal
        url = f'https://www.cnbcindonesia.com/search?query=PT%20Telkom%20Indonesia%20&lpar;Persero&rpar;%20Tbk&fromdate={start_date}&todate={end_date}&kanal=&tipe=&page={page}'
        ge = req.get(url, headers=hades).text
        sop = bs(ge, 'lxml')

        # Mencari semua artikel
        articles = sop.find_all('article')
        for article in articles:
            try:
                # Mengambil link artikel
                link = article.find('a')['href']

                # Mengambil judul artikel
                headline = article.find('h2').text.strip()

                # Mengakses halaman artikel untuk mengambil tanggal dan konten
                ge_ = req.get(link, headers=hades).text
                sop_ = bs(ge_, 'lxml')

                # Mengambil tanggal dari div id="detailHead"
                detail_head = sop_.find('div', id='detailHead')
                if detail_head:
                    date_element = detail_head.find('div', class_='text-cm text-gray')
                    date = date_element.text.strip() if date_element else 'Tanggal tidak ditemukan'
                else:
                    date = 'Tanggal tidak ditemukan'

                # Mengambil konten artikel dari struktur yang baru
                content_container = sop_.find('div', class_='flex gap-4')
                if content_container:
                    # Mencari elemen dengan class "detail-text min-w-0" atau "detail-text min-w-0 border-b"
                    detail_text = content_container.find('div', class_=lambda x: x and 'detail-text min-w-0' in x)
                    if detail_text:
                        # Mencari konten di dalam elemen yang ditemukan
                        content = detail_text.find('div', class_='detail-text')
                        if content:
                            # Mengambil semua paragraf dan menggabungkannya
                            content_ = ''.join([p.text for p in content.find_all('p')]).replace('\n', '').replace('ADVERTISEMENT', '').replace('SCROLL TO RESUME CONTENT', '')
                        else:
                            content_ = 'Konten tidak ditemukan'
                    else:
                        content_ = 'Konten tidak ditemukan'
                else:
                    content_ = 'Konten tidak ditemukan'

                # Menampilkan progress
                print(f'done[{a}] > {headline[0:50]}')
                a += 1

                # Menyimpan data ke dalam file CSV
                with open(csv_file, 'a', newline='', encoding='utf-8') as file:
                    wr = csv.writer(file, delimiter=',')
                    # Jika file belum ada, tulis header terlebih dahulu
                    if not file_exists:
                        wr.writerow(['headline', 'date', 'link', 'content'])
                        file_exists = True  # Set file_exists menjadi True setelah menulis header
                    # Tulis data artikel
                    wr.writerow([headline, date, link, content_])
            except Exception as e:
                print(f"Error pada artikel {a}: {e}")
                a += 1

In [ ]:
# Menjalankan fungsi scrape
scrape_cnbc(5000)

done[1] > IHSG Kurang Gairah, Saham Bank Raksasa Masih Terte
done[2] > IHSG Rebound! tapi Tekanan Jual Asing Masih Ada
done[3] > Duh! Ada Kabar Buruk dari AS, Pasar Bakal Keguncan
done[4] > Ditopang Rebound Saham Bank, IHSG Cerah Bergairah 
done[5] > Saham BBRI Bangkit, IHSG Balik ke 7.000-an,
done[6] > IHSG Longsor, Asing Net Sell Besar, Saham Ini Bany
done[7] > IHSG Ambruk 1,4%, Ternyata Ini Penyebabnya
done[8] > IHSG Ambruk 5 Hari Beruntun! Asing Lepas Saham Big
done[9] > Sabda Fed, Bakal Jadi Gempa Baru Buat IHSG & Rupia
done[10] > IHSG Merana Lagi, Nyaris Dekati 7.000-an
done[11] > IHSG Ambruk Lagi, Asing Lepas Triliunan Rupiah di 
done[12] > IHSG Terkapar, Saham Bank Raksasa Masih Tertekan
done[13] > IHSG Merana Balik Ke 7.200, Asing Kompak Lego Saha
done[14] > Banjir Stimulus untuk Dongkrak Daya Beli, Saatnya 
done[15] > IHSG Merosot Nyaris 1% ke 7.200, Ini Penyebabnya
done[16] > Begini Reaksi Bos Telkom Usai XL-Smartfren Resmi M
done[17] > IHSG Ambles, Sektor & Saham Ini Biang 